In [ ]:
from pathlib import Path

import numpy as np
import rasterio
from rasterio.enums import Resampling
from tqdm import tqdm
import subprocess

DIR = ...

image_paths = list(Path(DIR).glob("*.tif"))

global_min = np.inf
global_max = -np.inf

# First find the global min/max
for image_path in tqdm(image_paths):
    with rasterio.open(image_path) as src:
        data = src.read()

    min_val = np.nanmin(data)
    max_val = np.nanmax(data)
    global_min = min(min_val, global_min)
    global_max = max(max_val, global_max)

global_min, global_max

In [ ]:
out_dir = f"{DIR}_scaled"
Path(out_dir).mkdir(exist_ok=True)

# Now scale
for image_path in tqdm(image_paths):
    with rasterio.open(image_path) as src:
        data = src.read()
        meta = src.meta

    # Uncomment to save as uint8 format with nan set to 0
    data = (data - global_min) / (global_max - global_min)
    data = (data * 255).astype(np.uint8)
    data = np.nan_to_num(data, nan=0)
    meta.update(dtype=data.dtype, nodata=0)

    # Uncomment to save as float32 format with nan but scaled to uint8
    # data = (data - global_min) / (global_max - global_min)
    # data = np.clip(np.round(data * 255), 0, 255)
    # data = data.astype(np.float32)

    save_path = f"{out_dir}/{image_path.name}"
    with rasterio.open(save_path, "w", **meta) as dst:
        dst.write(data)
        dst.build_overviews([2, 4, 8, 16, 32], Resampling.average)
        dst.update_tags(ns="rio_overview", resampling="average")

In [ ]:
def gdalwarp_cog(input_folder, output_filepath):
    tif_files = [str(path) for path in Path(input_folder).glob("*.tif")]
    cmd = (
        [
            "uv",
            "run",
            "gdalwarp",
            "-t_srs",
            "EPSG:3857",
            "-of",
            "COG",
            "-co",
            "COMPRESS=ZSTD",
            "-co",
            "TILED=YES",
            "-co",
            "BIGTIFF=YES",
            "-r",
            "bilinear",
        ]
        + tif_files
        + [output_filepath]
    )
    subprocess.run(cmd, check=True)


gdalwarp_cog(
    ...,
    ...,
)